# Custom Components

This is a guide on how to build a simple app and custom component spec
and launch it via two different schedulers.

See the [Quickstart Guide](quickstart.md) for installation and basic usage.

## Hello World

Lets start off with writing a simple "Hello World" python app. This is just a
normal python program and can contain anything you'd like.

<div class="admonition note">
<div class="admonition-title">Note</div>
This example uses Jupyter Notebook `%%writefile` to create local files for
example purposes. Under normal usage you would have these as standalone files.
</div>

In [1]:
%%writefile my_app.py

import sys
import argparse

def main(user: str) -> None:
    print(f"Hello, {user}!")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description="Hello world app"
    )
    parser.add_argument(
        "--user",
        type=str,
        help="the person to greet",
        required=True,
    )
    args = parser.parse_args(sys.argv[1:])

    main(args.user)

Overwriting my_app.py


Now that we have an app we can write the component file for it. This
function allows us to reuse and share our app in a user friendly way.

We can use this component from the `torchx` cli or programmatically as part of a
pipeline.

In [2]:
%%writefile my_component.py

import torchx.specs as specs

def greet(user: str, image: str = "my_app:latest") -> specs.AppDef:
    return specs.AppDef(
        name="hello_world",
        roles=[
            specs.Role(
                name="greeter",
                image=image,
                entrypoint="python",
                args=[
                    "-m", "my_app",
                    "--user", user,
                ],
            )
        ],
    )

Overwriting my_component.py


We can execute our component via `torchx run`. The
`local_cwd` scheduler executes the component relative to the current directory.

In [3]:
%%sh
torchx run --scheduler local_cwd my_component.py:greet --user "your name"

torchx 2023-10-17 17:32:51 INFO     loaded configs from /home/ubuntu/torchx/docs/source/.torchxconfig


torchx 2023-10-17 17:32:51 INFO     Tracker configurations: {}


torchx 2023-10-17 17:32:51 INFO     Log directory not set in scheduler cfg. Creating a temporary log dir that will be deleted on exit. To preserve log directory set the `log_dir` cfg option


torchx 2023-10-17 17:32:51 INFO     Log directory is: /tmp/torchx_onri392j


torchx 2023-10-17 17:32:51 INFO     Waiting for the app to finish...


greeter/0 Hello, your name!


torchx 2023-10-17 17:32:53 INFO     Job finished: SUCCEEDED


local_cwd://torchx/hello_world-vs1snfjpdr160


If we want to run in other environments, we can build a Docker container so we
can run our component in Docker enabled environments such as Kubernetes or via
the local Docker scheduler.

<div class="admonition note">
<div class="admonition-title">Note</div>
This requires Docker installed and won't work in environments such as Google
Colab. If you have not done so already follow the install instructions on:
[https://docs.docker.com/get-docker/](https://docs.docker.com/get-docker/)</a>
</div>

In [4]:
%%writefile Dockerfile.custom

FROM ghcr.io/pytorch/torchx:0.1.0rc1

ADD my_app.py .

Overwriting Dockerfile.custom


Once we have the Dockerfile created we can create our docker image.

In [5]:
%%sh
docker build -t my_app:latest -f Dockerfile.custom .

DEPRECATED: The legacy builder is deprecated and will be removed in a future release.


            Install the buildx component to build images with BuildKit:


            https://docs.docker.com/go/buildx/


Step 1/2 : FROM ghcr.io/pytorch/torchx:0.1.0rc1


 ---> 3dbec59e8049


Step 2/2 : ADD my_app.py .


 ---> Using cache


 ---> 5a979ce3c50c


Successfully built 5a979ce3c50c


Successfully tagged my_app:latest


We can then launch it on the local scheduler.

In [6]:
%%sh
torchx run --scheduler local_docker my_component.py:greet --image "my_app:latest" --user "your name"

torchx 2023-10-17 17:32:53 INFO     loaded configs from /home/ubuntu/torchx/docs/source/.torchxconfig


torchx 2023-10-17 17:32:53 INFO     Tracker configurations: {}


torchx 2023-10-17 17:32:53 INFO     Checking for changes in workspace `file:///home/ubuntu/torchx/docs/source`...


torchx 2023-10-17 17:32:53 INFO     To disable workspaces pass: --workspace="" from CLI or workspace=None programmatically.


torchx 2023-10-17 17:32:53 INFO     Workspace `file:///home/ubuntu/torchx/docs/source` resolved to filesystem path `/home/ubuntu/torchx/docs/source`


...

torchx 2023-10-17 17:32:55 INFO     Built new image `sha256:5267425173c914ada6326dae6c2a48aa396a04baf113395d87e8b6c68d344029` based on original image `my_app:latest` and changes in workspace `file:///home/ubuntu/torchx/docs/source` for role[0]=greeter.


torchx 2023-10-17 17:32:55 INFO     Waiting for the app to finish...


greeter/0 Hello, your name!


torchx 2023-10-17 17:32:56 INFO     Job finished: SUCCEEDED


local_docker://torchx/hello_world-ph6ccqd0z056k


If you have a Kubernetes cluster you can use the [Kubernetes scheduler](schedulers/kubernetes.rst) to launch
this on the cluster instead.


```sh
$ docker push my_app:latest
$ torchx run --scheduler kubernetes my_component.py:greet --image "my_app:latest" --user "your name"
```

## Builtins

TorchX also provides a number of builtin components with premade images. You can discover
them via:

In [7]:
%%sh
torchx builtins

Found 11 builtin components:


  1. dist.ddp


  2. dist.spmd


  3. metrics.tensorboard


  4. serve.torchserve


...

  7. utils.copy


  8. utils.echo


  9. utils.python


 10. utils.sh


 11. utils.touch


You can use these either from the CLI, from a pipeline or programmatically like
you would any other component.

In [8]:
%%sh
torchx run utils.echo --msg "Hello :)"

torchx 2023-10-17 17:32:58 INFO     loaded configs from /home/ubuntu/torchx/docs/source/.torchxconfig


torchx 2023-10-17 17:32:59 INFO     Tracker configurations: {}


torchx 2023-10-17 17:32:59 INFO     Checking for changes in workspace `file:///home/ubuntu/torchx/docs/source`...


torchx 2023-10-17 17:32:59 INFO     To disable workspaces pass: --workspace="" from CLI or workspace=None programmatically.


torchx 2023-10-17 17:32:59 INFO     Workspace `file:///home/ubuntu/torchx/docs/source` resolved to filesystem path `/home/ubuntu/torchx/docs/source`


...

torchx 2023-10-17 17:33:00 INFO     Built new image `sha256:5267425173c914ada6326dae6c2a48aa396a04baf113395d87e8b6c68d344029` based on original image `ghcr.io/pytorch/torchx:0.6.0` and changes in workspace `file:///home/ubuntu/torchx/docs/source` for role[0]=echo.


torchx 2023-10-17 17:33:00 INFO     Waiting for the app to finish...


torchx 2023-10-17 17:33:00 INFO     Job finished: SUCCEEDED


echo/0 Hello :)


local_docker://torchx/echo-h5dlkcgmq3342
